## IMPORT REQUIRED MODULES

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime

## Overall Gainers

In [2]:
# Step 1: Set up the request
url = 'https://m.economictimes.com/stocks/marketstats/top-gainers/nse'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36',
    'Referer': 'https://www.google.com'
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# Step 2: Extract column names
columns = [div.get_text(strip=True) for div in soup.find_all("div", class_="MarketTable_theading__R8YA5")]

# Step 3: Get both tbody sections
tbody_sections = soup.find_all('tbody')
tbody1 = tbody_sections[0]
tbody2 = tbody_sections[1]

# Step 4: Extract first 3 columns from tbody1
part1 = []
for row in tbody1.find_all('tr'):
    cells = row.find_all('td')
    part1.append([cell.get_text(strip=True) for cell in cells[:3]])

# Step 5: Extract remaining columns from tbody2
part2 = []
for row in tbody2.find_all('tr'):
    cells = row.find_all('td')
    part2.append([cell.get_text(strip=True) for cell in cells])

# Step 6: Combine both parts
combined_data = [p1 + p2 for p1, p2 in zip(part1, part2)]

# Step 7: Fix column mismatch if needed
if len(columns) < len(combined_data[0]):
    columns.append("Extra Column")  # Add placeholder for extra value

# Step 8: Create DataFrame
all_gainers = pd.DataFrame(combined_data, columns=columns)


all_gainers = all_gainers.drop(columns=['Extra Column'])

all_gainers["Date"] = datetime.today().date()

# Reanme the columns
all_gainers = all_gainers.rename(columns={
    'Company Name': 'company_name',
    'Current Price (Rs)': 'current_price_rs',
    'Price Change': 'price_change',
    '% Change': 'percent_change',
    'Stock Score': 'stock_score',
    'Potential Upside': 'potential_upside',
    'Volume': 'volume',
    'Total Asset Turnover': 'total_asset_turnover',
    'Market Cap (Rs Cr)': 'market_cap_rs_cr',
    '1W Returns': 'one_week_returns',
    '1M Returns': 'one_month_returns',
    '3M Returns': 'three_month_returns',
    '6M Returns': 'six_month_returns',
    '1Y Returns': 'one_year_returns',
    '3Y Returns': 'three_year_returns',
    'YTD Returns': 'ytd_returns',
    'Date': 'date'
})


# Rename the star-column to your canonical name
all_gainers = all_gainers.rename(columns={'Potential Upside*': 'potential_upside'})

# Now this will work
all_gainers = all_gainers.drop(columns=['potential_upside', 'stock_score'])
all_gainers = all_gainers.head(10)
all_gainers 


,company_name,current_price_rs,price_change,percent_change,volume,total_asset_turnover,market_cap_rs_cr,one_week_returns,one_month_returns,three_month_returns,six_month_returns,one_year_returns,three_year_returns,ytd_returns,date
0,Niraj Ispat Ind.,257.41,42.90,20.00%,"3,287",0.21,-,-,-,-,-,-,-,-,2026-02-06
1,Gorani Ind,63.61,10.60,20.00%,"2,678",1.46,34.10,13.94%,9.62%,-11.16%,-6.92%,-38.78%,-64.14%,2.58%,2026-02-06
2,Stratmont Industries,90.01,15.00,20.00%,"87,078",1.58,257.00,66.69%,46.10%,31.27%,-17.82%,-4.16%,342.54%,49.59%,2026-02-06
3,Mega Corp,2.56,0.42,19.60%,"3,74,885",0.09,51.20,19.63%,7.56%,-1.16%,1.99%,41.44%,39.13%,16.89%,2026-02-06
4,IZMO,841.45,131.00,18.50%,"20,75,758",0.56,"1,258",40.41%,5.33%,-6.87%,133.74%,95.16%,"1,088%",4.12%,2026-02-06
5,Apollo Pipes,313.40,48.00,18.10%,"42,20,194",1.05,"1,380",16.83%,6.22%,6.98%,-22.25%,-24.89%,-37.45%,6.78%,2026-02-06
6,Loyal Textiles,314.79,46.70,17.40%,"1,80,059",0.70,152.00,58.18%,47.82%,39.26%,-10.87%,5.72%,-55.06%,47.13%,2026-02-06
7,TPI India,19.51,2.81,16.80%,"4,10,433",3.31,83.80,21.94%,21.94%,26.94%,24.82%,8.99%,436.83%,26.94%,2026-02-06
8,Gajanan Sec. Serv.,60.78,8.28,15.80%,"12,619",-,18.90,7.14%,6.67%,0.66%,-16.19%,-49.41%,408.53%,14.57%,2026-02-06
9,Luharuka Media & Infra,2.60,0.35,15.60%,"2,36,558",0.12,55.80,12.55%,-22.62%,-35.80%,-38.68%,-49.71%,-28.18%,-24.42%,2026-02-06


In [3]:

print(all_gainers.columns.tolist())


['company_name', 'current_price_rs', 'price_change', 'percent_change', 'volume', 'total_asset_turnover', 'market_cap_rs_cr', 'one_week_returns', 'one_month_returns', 'three_month_returns', 'six_month_returns', 'one_year_returns', 'three_year_returns', 'ytd_returns', 'date']


## Overall Loosers

In [4]:
url = 'https://m.economictimes.com/stocks/marketstats/top-losers/nse'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36',
    'Referer': 'https://www.google.com'
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# Step 2: Extract column names
columns = [div.get_text(strip=True) for div in soup.find_all("div", class_="MarketTable_theading__R8YA5")]

# Step 3: Get both tbody sections
tbody_sections = soup.find_all('tbody')
tbody1 = tbody_sections[0]
tbody2 = tbody_sections[1]

# Step 4: Extract first 3 columns from tbody1
part1 = []
for row in tbody1.find_all('tr'):
    cells = row.find_all('td')
    part1.append([cell.get_text(strip=True) for cell in cells[:3]])

# Step 5: Extract remaining columns from tbody2
part2 = []
for row in tbody2.find_all('tr'):
    cells = row.find_all('td')
    part2.append([cell.get_text(strip=True) for cell in cells])

# Step 6: Combine both parts
combined_data = [p1 + p2 for p1, p2 in zip(part1, part2)]

# Step 7: Fix column mismatch if needed
if len(columns) < len(combined_data[0]):
    columns.append("Extra Column")  # Add placeholder for extra value

# Step 8: Create DataFrame
all_loosers = pd.DataFrame(combined_data, columns=columns)


all_loosers = all_loosers.drop(columns=['Extra Column'])

all_loosers["Date"] = datetime.today().date()

all_loosers = all_loosers.rename(columns={
    'Company Name': 'company_name',
    'Current Price (Rs)': 'current_price_rs',
    'Price Change': 'price_change',
    '% Change': 'percent_change',
    'Stock Score': 'stock_score',
    'Potential Upside': 'potential_upside',
    'Volume': 'volume',
    'Total Asset Turnover': 'total_asset_turnover',
    'Market Cap (Rs Cr)': 'market_cap_rs_cr',
    '1W Returns': 'one_week_returns',
    '1M Returns': 'one_month_returns',
    '3M Returns': 'three_month_returns',
    '6M Returns': 'six_month_returns',
    '1Y Returns': 'one_year_returns',
    '3Y Returns': 'three_year_returns',
    'YTD Returns': 'ytd_returns',
    'Date': 'date'
})

# Rename the star-column to your canonical name
all_loosers = all_loosers.rename(columns={'Potential Upside*': 'potential_upside'})

# Now this will work
all_loosers = all_loosers.drop(columns=['potential_upside', 'stock_score'])

all_loosers = all_loosers.head(10)

all_loosers

,company_name,current_price_rs,price_change,percent_change,volume,total_asset_turnover,market_cap_rs_cr,one_week_returns,one_month_returns,three_month_returns,six_month_returns,one_year_returns,three_year_returns,ytd_returns,date
0,CL Educate,59.08,-14.77,-20.00%,"3,58,493",0.56,320.00,-20.18%,-28.79%,-33.38%,-28.28%,-52.20%,0.56%,-32.93%,2026-02-06
1,Vivid Mercantile,7.08,-1.45,-16.99%,"46,13,664",0.23,71.00,3.81%,6.63%,52.92%,4.27%,-0.84%,129.13%,8.76%,2026-02-06
2,Veer Global Infraconstruction,91.55,-17.14,-15.76%,38,0.15,149.00,-7.80%,-23.71%,-28.81%,-32.91%,-34.61%,32.95%,-23.07%,2026-02-06
3,Olympic Cards,2.80,-0.50,-15.15%,502,0.20,4.57,-3.78%,-13.85%,-2.44%,-6.35%,-20.00%,-14.89%,-11.95%,2026-02-06
4,Trio Mercantile,0.70,-0.10,-12.50%,"4,24,047",0.06,4.76,12.90%,-12.50%,-9.09%,27.27%,-36.36%,-14.63%,-1.41%,2026-02-06
5,Svaraj Trading,6.76,-0.96,-12.43%,"22,073",0.01,9.97,-8.65%,-9.50%,-23.01%,-28.39%,-26.52%,6.12%,-12.21%,2026-02-06
6,Maheshwari Logistics,46.27,-6.53,-12.37%,"4,36,142",1.68,137.00,4.21%,-1.70%,-15.57%,-22.59%,-25.62%,-25.67%,-2.24%,2026-02-06
7,Paragon Finance,45.32,-6.26,-12.13%,58,0.07,19.30,-5.96%,-4.69%,-18.19%,-16.84%,-21.11%,86.50%,-6.96%,2026-02-06
8,Solara Active Pharma,463.85,-62.61,-11.90%,"4,09,808",0.56,"2,268",-6.98%,-16.98%,-16.32%,-19.65%,-13.33%,20.92%,-16.38%,2026-02-06
9,Rose Merc. Ltd.,68.13,-8.69,-11.31%,"20,949",1.42,38.80,-4.20%,-18.32%,10.12%,-30.16%,-10.37%,131.34%,-17.94%,2026-02-06


## NSE Gainers

In [5]:
# Step 1: Set up the request
url = 'https://m.economictimes.com/stocks/marketstats/top-gainers'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36',
    'Referer': 'https://www.google.com'
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# Step 2: Extract column names
columns = [div.get_text(strip=True) for div in soup.find_all("div", class_="MarketTable_theading__R8YA5")]

# Step 3: Get both tbody sections
tbody_sections = soup.find_all('tbody')
tbody1 = tbody_sections[0]
tbody2 = tbody_sections[1]

# Step 4: Extract first 3 columns from tbody1
part1 = []
for row in tbody1.find_all('tr'):
    cells = row.find_all('td')
    part1.append([cell.get_text(strip=True) for cell in cells[:3]])

# Step 5: Extract remaining columns from tbody2
part2 = []
for row in tbody2.find_all('tr'):
    cells = row.find_all('td')
    part2.append([cell.get_text(strip=True) for cell in cells])

# Step 6: Combine both parts
combined_data = [p1 + p2 for p1, p2 in zip(part1, part2)]

# Step 7: Fix column mismatch if needed
if len(columns) < len(combined_data[0]):
    columns.append("Extra Column")  # Add placeholder for extra value

# Step 8: Create DataFrame
nse_gainers = pd.DataFrame(combined_data, columns=columns)


nse_gainers = nse_gainers.drop(columns=['Extra Column'])

nse_gainers["Date"] = datetime.today().date()

nse_gainers = nse_gainers.rename(columns={
    'Company Name': 'company_name',
    'Current Price (Rs)': 'current_price_rs',
    'Price Change': 'price_change',
    '% Change': 'percent_change',
    'Stock Score': 'stock_score',
    'Potential Upside': 'potential_upside',
    'Volume': 'volume',
    'Total Asset Turnover': 'total_asset_turnover',
    'Market Cap (Rs Cr)': 'market_cap_rs_cr',
    '1W Returns': 'one_week_returns',
    '1M Returns': 'one_month_returns',
    '3M Returns': 'three_month_returns',
    '6M Returns': 'six_month_returns',
    '1Y Returns': 'one_year_returns',
    '3Y Returns': 'three_year_returns',
    'YTD Returns': 'ytd_returns',
    'Date': 'date'
})


nse_gainers = nse_gainers.rename(columns={'Potential Upside*': 'potential_upside'})

# Now this will work
nse_gainers = nse_gainers.drop(columns=['potential_upside', 'stock_score'])


nse_gainers = nse_gainers.head(10)
nse_gainers

,company_name,current_price_rs,price_change,percent_change,volume,total_asset_turnover,market_cap_rs_cr,one_week_returns,one_month_returns,three_month_returns,six_month_returns,one_year_returns,three_year_returns,ytd_returns,date
0,ABB Power,"21,850","2,651",13.80%,"9,93,823",0.96,"97,484",15.95%,16.04%,8.23%,7.11%,78.05%,603.59%,18.34%,2026-02-06
1,Concord Biotech Ltd.,"1,327",159.00,13.60%,"38,30,074",0.64,"13,885",12.93%,-1.85%,-5.97%,-19.72%,-43.34%,-,-0.68%,2026-02-06
2,Godfrey Philips,"2,188",206.00,10.40%,"72,90,096",0.88,"34,133",7.51%,0.34%,-27.62%,-39.52%,42.73%,264.33%,-4.46%,2026-02-06
3,MRF,"1,46,455","11,485",8.51%,"64,381",1.00,"62,114",10.81%,-2.49%,-6.87%,0.05%,28.32%,58.03%,-3.35%,2026-02-06
4,Data Patterns (India),"2,724",200.00,7.94%,"35,63,917",0.40,"15,248",1.94%,1.68%,4.20%,4.92%,38.69%,83.89%,4.71%,2026-02-06
5,FSN E-Comm (Nykaa),276.99,18.70,7.24%,"5,76,86,215",2.15,"79,289",16.63%,2.95%,12.53%,32.77%,59.35%,100.79%,4.23%,2026-02-06
6,LIC,899.00,59.30,7.06%,"1,58,17,666",0.16,"5,70,420",9.38%,6.09%,0.64%,1.02%,8.81%,50.33%,5.75%,2026-02-06
7,Mahanagar Gas,"1,139",67.40,6.30%,"12,12,079",0.93,"11,385",9.48%,7.28%,-6.50%,-10.84%,-14.12%,30.68%,2.17%,2026-02-06
8,Siemens Energy India,"2,741",142.00,5.46%,"10,91,392",-,"97,598",10.19%,4.52%,-13.97%,-13.32%,-,-,7.94%,2026-02-06
9,Poonawalla Fincorp,431.50,21.50,5.24%,"21,40,087",0.14,"35,065",8.15%,-9.96%,-6.78%,-2.02%,35.39%,47.75%,-10.70%,2026-02-06


# NSE Loosers

In [6]:
url = 'https://m.economictimes.com/stocks/marketstats/top-losers'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36',
    'Referer': 'https://www.google.com'
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# Step 2: Extract column names
columns = [div.get_text(strip=True) for div in soup.find_all("div", class_="MarketTable_theading__R8YA5")]

# Step 3: Get both tbody sections
tbody_sections = soup.find_all('tbody')
tbody1 = tbody_sections[0]
tbody2 = tbody_sections[1]

# Step 4: Extract first 3 columns from tbody1
part1 = []
for row in tbody1.find_all('tr'):
    cells = row.find_all('td')
    part1.append([cell.get_text(strip=True) for cell in cells[:3]])

# Step 5: Extract remaining columns from tbody2
part2 = []
for row in tbody2.find_all('tr'):
    cells = row.find_all('td')
    part2.append([cell.get_text(strip=True) for cell in cells])

# Step 6: Combine both parts
combined_data = [p1 + p2 for p1, p2 in zip(part1, part2)]

# Step 7: Fix column mismatch if needed
if len(columns) < len(combined_data[0]):
    columns.append("Extra Column")  # Add placeholder for extra value

# Step 8: Create DataFrame
nse_loosers = pd.DataFrame(combined_data, columns=columns)


nse_loosers = nse_loosers.drop(columns=['Extra Column'])

nse_loosers["Date"] = datetime.today().date()

nse_loosers = nse_loosers.rename(columns={
    'Company Name': 'company_name',
    'Current Price (Rs)': 'current_price_rs',
    'Price Change': 'price_change',
    '% Change': 'percent_change',
    'Stock Score': 'stock_score',
    'Potential Upside': 'potential_upside',
    'Volume': 'volume',
    'Total Asset Turnover': 'total_asset_turnover',
    'Market Cap (Rs Cr)': 'market_cap_rs_cr',
    '1W Returns': 'one_week_returns',
    '1M Returns': 'one_month_returns',
    '3M Returns': 'three_month_returns',
    '6M Returns': 'six_month_returns',
    '1Y Returns': 'one_year_returns',
    '3Y Returns': 'three_year_returns',
    'YTD Returns': 'ytd_returns',
    'Date': 'date'
})


nse_loosers = nse_loosers.rename(columns={'Potential Upside*': 'potential_upside'})

# Now this will work
nse_loosers = nse_loosers.drop(columns=['potential_upside', 'stock_score'])


nse_loosers = nse_loosers.head(10)

nse_loosers

,company_name,current_price_rs,price_change,percent_change,volume,total_asset_turnover,market_cap_rs_cr,one_week_returns,one_month_returns,three_month_returns,six_month_returns,one_year_returns,three_year_returns,ytd_returns,date
0,Poly Medicure,"1,389",-113.00,-7.53%,"5,04,104",0.66,"14,083",-8.65%,-22.28%,-26.75%,-29.02%,-42.91%,66.88%,-20.26%,2026-02-06
1,BEML,"1,617",-129.00,-7.38%,"17,32,960",0.71,"13,471",-9.89%,-12.66%,-18.59%,-18.16%,-1.34%,137.42%,-12.45%,2026-02-06
2,eClerx Services,"4,186",-311.00,-6.93%,"1,63,725",1.11,"19,684",-10.00%,-12.84%,-8.40%,5.93%,31.61%,178.23%,-13.19%,2026-02-06
3,Minda Ind.,"1,158",-72.80,-5.92%,"23,82,069",1.55,"66,823",-2.06%,-12.33%,-6.14%,6.59%,12.41%,133.84%,-10.00%,2026-02-06
4,Aavas Financiers,"1,302",-77.41,-5.62%,"10,86,521",0.13,"10,304",-10.94%,-11.00%,-18.86%,-23.66%,-24.53%,-34.58%,-10.81%,2026-02-06
5,Birlasoft,435.70,-24.86,-5.40%,"15,64,038",1.28,"12,143",4.22%,3.42%,16.83%,12.88%,-17.78%,53.52%,0.86%,2026-02-06
6,Siemens,"3,139",-161.00,-4.88%,"8,50,780",0.76,"1,13,111",2.42%,1.54%,3.01%,2.99%,-43.69%,3.18%,2.75%,2026-02-06
7,Suven Pharma,339.45,-17.36,-4.87%,"9,40,350",0.45,"12,986",-10.98%,-32.42%,-53.10%,-63.74%,-70.88%,-30.64%,-35.16%,2026-02-06
8,Aegis Vopak Terminal,217.81,-11.08,-4.85%,"8,30,016",0.12,"24,133",-3.97%,-9.09%,-15.66%,-10.50%,-,-,-13.64%,2026-02-06
9,Hind Copper,578.20,-29.10,-4.80%,"2,77,05,587",0.61,"56,073",-15.46%,2.58%,79.91%,136.84%,140.92%,448.06%,10.94%,2026-02-06


# BSE Gainers

In [7]:
# Step 1: Set up the request
url = 'https://m.economictimes.com/stocks/marketstats/top-gainers/sensex'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36',
    'Referer': 'https://www.google.com'
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# Step 2: Extract column names
columns = [div.get_text(strip=True) for div in soup.find_all("div", class_="MarketTable_theading__R8YA5")]

# Step 3: Get both tbody sections
tbody_sections = soup.find_all('tbody')
tbody1 = tbody_sections[0]
tbody2 = tbody_sections[1]

# Step 4: Extract first 3 columns from tbody1
part1 = []
for row in tbody1.find_all('tr'):
    cells = row.find_all('td')
    part1.append([cell.get_text(strip=True) for cell in cells[:3]])

# Step 5: Extract remaining columns from tbody2
part2 = []
for row in tbody2.find_all('tr'):
    cells = row.find_all('td')
    part2.append([cell.get_text(strip=True) for cell in cells])

# Step 6: Combine both parts
combined_data = [p1 + p2 for p1, p2 in zip(part1, part2)]

# Step 7: Fix column mismatch if needed
if len(columns) < len(combined_data[0]):
    columns.append("Extra Column")  # Add placeholder for extra value

# Step 8: Create DataFrame
bse_gainers = pd.DataFrame(combined_data, columns=columns)


bse_gainers = bse_gainers.drop(columns=['Extra Column'])

bse_gainers["Date"] = datetime.today().date()

bse_gainers = bse_gainers.rename(columns={
    'Company Name': 'company_name',
    'Current Price (Rs)': 'current_price_rs',
    'Price Change': 'price_change',
    '% Change': 'percent_change',
    'Stock Score': 'stock_score',
    'Potential Upside': 'potential_upside',
    'Volume': 'volume',
    'Total Asset Turnover': 'total_asset_turnover',
    'Market Cap (Rs Cr)': 'market_cap_rs_cr',
    '1W Returns': 'one_week_returns',
    '1M Returns': 'one_month_returns',
    '3M Returns': 'three_month_returns',
    '6M Returns': 'six_month_returns',
    '1Y Returns': 'one_year_returns',
    '3Y Returns': 'three_year_returns',
    'YTD Returns': 'ytd_returns',
    'Date': 'date'
})

bse_gainers = bse_gainers.rename(columns={'Potential Upside*': 'potential_upside'})

# Now this will work
bse_gainers = bse_gainers.drop(columns=['potential_upside', 'stock_score'])

bse_gainers = bse_gainers.head(10)
bse_gainers

,company_name,current_price_rs,price_change,percent_change,volume,total_asset_turnover,market_cap_rs_cr,one_week_returns,one_month_returns,three_month_returns,six_month_returns,one_year_returns,three_year_returns,ytd_returns,date
0,ITC,326.35,16.20,5.21%,"6,80,46,151",0.84,"4,08,194",1.13%,-4.86%,-20.05%,-20.92%,-26.14%,-15.02%,-10.46%,2026-02-06
1,Kotak Bank,422.35,13.60,3.33%,"2,12,27,722",-,"84,010",3.52%,-1.61%,1.37%,5.46%,10.22%,20.88%,-4.78%,2026-02-06
2,HUL,"2,424",69.80,2.97%,"14,21,995",0.80,"5,69,588",2.16%,-0.02%,-0.48%,-4.39%,2.17%,-8.09%,4.36%,2026-02-06
3,Bajaj Finance,982.00,17.30,1.79%,"1,37,42,088",0.17,"6,10,861",5.58%,0.45%,-5.78%,11.98%,15.40%,60.82%,0.88%,2026-02-06
4,Bharti Airtel,"2,023",30.60,1.54%,"1,38,70,286",0.36,"12,42,281",3.54%,-3.18%,-2.70%,5.63%,25.85%,158.27%,-3.41%,2026-02-06
5,Power Grid,292.80,3.45,1.20%,"1,99,45,330",0.18,"2,72,275",14.13%,9.01%,8.35%,2.79%,4.13%,79.84%,9.73%,2026-02-06
6,Titan Company,"4,141",43.20,1.06%,"8,88,561",1.67,"3,67,632",4.11%,0.71%,9.71%,21.16%,21.41%,68.47%,2.26%,2026-02-06
7,Bajaj Finserv,"2,021",20.70,1.04%,"12,57,201",0.22,"3,23,429",3.67%,-1.00%,-1.89%,5.21%,13.21%,50.72%,-0.63%,2026-02-06
8,ICICI Bank,"1,408",11.50,0.83%,"1,08,50,946",-,"10,05,456",3.77%,-0.36%,6.49%,-2.60%,10.51%,64.71%,5.09%,2026-02-06
9,Axis Bank,"1,342",11.00,0.83%,"58,76,574",-,"4,16,578",-2.10%,3.69%,9.21%,25.27%,31.44%,52.00%,5.27%,2026-02-06


# BSE Loosers

In [8]:
url = 'https://m.economictimes.com/stocks/marketstats/top-losers/sensex'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36',
    'Referer': 'https://www.google.com'
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# Step 2: Extract column names
columns = [div.get_text(strip=True) for div in soup.find_all("div", class_="MarketTable_theading__R8YA5")]

# Step 3: Get both tbody sections
tbody_sections = soup.find_all('tbody')
tbody1 = tbody_sections[0]
tbody2 = tbody_sections[1]

# Step 4: Extract first 3 columns from tbody1
part1 = []
for row in tbody1.find_all('tr'):
    cells = row.find_all('td')
    part1.append([cell.get_text(strip=True) for cell in cells[:3]])

# Step 5: Extract remaining columns from tbody2
part2 = []
for row in tbody2.find_all('tr'):
    cells = row.find_all('td')
    part2.append([cell.get_text(strip=True) for cell in cells])

# Step 6: Combine both parts
combined_data = [p1 + p2 for p1, p2 in zip(part1, part2)]

# Step 7: Fix column mismatch if needed
if len(columns) < len(combined_data[0]):
    columns.append("Extra Column")  # Add placeholder for extra value

# Step 8: Create DataFrame
bse_loosers = pd.DataFrame(combined_data, columns=columns)


bse_loosers = bse_loosers.drop(columns=['Extra Column'])

bse_loosers["Date"] = datetime.today().date()

bse_loosers = bse_loosers.rename(columns={
    'Company Name': 'company_name',
    'Current Price (Rs)': 'current_price_rs',
    'Price Change': 'price_change',
    '% Change': 'percent_change',
    'Stock Score': 'stock_score',
    'Potential Upside': 'potential_upside',
    'Volume': 'volume',
    'Total Asset Turnover': 'total_asset_turnover',
    'Market Cap (Rs Cr)': 'market_cap_rs_cr',
    '1W Returns': 'one_week_returns',
    '1M Returns': 'one_month_returns',
    '3M Returns': 'three_month_returns',
    '6M Returns': 'six_month_returns',
    '1Y Returns': 'one_year_returns',
    '3Y Returns': 'three_year_returns',
    'YTD Returns': 'ytd_returns',
    'Date': 'date'
})


bse_loosers = bse_loosers.rename(columns={'Potential Upside*': 'potential_upside'})

# Now this will work
bse_loosers =bse_loosers.drop(columns=['potential_upside', 'stock_score'])

bse_loosers = bse_loosers.head(10)
bse_loosers

,company_name,current_price_rs,price_change,percent_change,volume,total_asset_turnover,market_cap_rs_cr,one_week_returns,one_month_returns,three_month_returns,six_month_returns,one_year_returns,three_year_returns,ytd_returns,date
0,Tech Mahindra,"1,616",-30.21,-1.84%,"19,88,405",1.21,"1,58,705",-7.07%,1.13%,14.59%,11.03%,-2.98%,61.13%,0.76%,2026-02-06
1,TCS,"2,940",-51.20,-1.72%,"46,86,333",1.67,"10,64,297",-5.84%,-9.65%,-2.30%,-2.99%,-27.95%,-14.98%,-8.86%,2026-02-06
2,Adani Ports SEZ,"1,550",-20.71,-1.32%,"44,27,976",0.24,"3,57,275",9.22%,5.26%,7.93%,13.43%,33.22%,184.30%,4.70%,2026-02-06
3,Eternal,283.55,-3.31,-1.16%,"2,99,27,803",0.69,"2,73,636",3.64%,1.61%,-7.23%,-5.12%,23.79%,493.82%,-0.09%,2026-02-06
4,Asian Paints,"2,405",-27.10,-1.12%,"10,43,921",1.12,"2,30,313",-1.12%,-15.63%,-7.75%,-3.62%,6.08%,-12.82%,-12.75%,2026-02-06
5,HCL Tech,"1,594",-16.30,-1.02%,"33,42,641",1.14,"4,32,477",-6.01%,-1.40%,4.41%,8.92%,-7.53%,39.10%,-2.50%,2026-02-06
6,Infosys,"1,506",-14.50,-0.96%,"1,38,99,913",1.14,"6,11,079",-8.16%,-6.52%,2.75%,4.90%,-21.33%,-4.00%,-7.53%,2026-02-06
7,HDFC Bank,941.00,-8.71,-0.92%,"1,81,19,162",-,"14,47,843",1.28%,-2.19%,-4.42%,-5.19%,7.93%,13.95%,-5.05%,2026-02-06
8,Trent,"4,095",-36.31,-0.88%,"11,15,473",2.07,"1,46,240",8.67%,1.64%,-12.08%,-23.20%,-22.04%,234.43%,-4.27%,2026-02-06
9,SBI,"1,066",-7.50,-0.70%,"70,40,278",-,"9,84,353",-1.00%,4.66%,11.00%,32.45%,41.76%,95.53%,8.29%,2026-02-06


In [9]:
import pandas as pd
from sqlalchemy import create_engine
from urllib.parse import quote_plus

# ✅ PostgreSQL connection details
db_user = 'postgres'
db_password = quote_plus('Manjuk@25465')  # Encode special characters
db_host = 'localhost'
db_port = '5432'
db_name = 'Stock data'

# 🔗 Create SQLAlchemy engine
engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')
print("✅ Connected to PostgreSQL")

# ✅ Dictionary of table names and corresponding DataFrames
stock_data = ['all_loosers', 'all_gainers', 'nse_gainers', 'nse_loosers', 'bse_gainers', 'bse_loosers']

for name in stock_data:
    try:
        df = globals().get(name)  # Get the actual DataFrame object
        if df is not None and not df.empty:
            df.to_sql(name, engine, if_exists='append', index=False)
            print(f"✅ Exported '{name}' to SQL successfully.")
        else:
            print(f"⚠️ Skipped '{name}' — DataFrame is empty or not found.")
    except Exception as e:
        print(f"❌ Failed to export '{name}': {e}")

✅ Connected to PostgreSQL
✅ Exported 'all_loosers' to SQL successfully.
✅ Exported 'all_gainers' to SQL successfully.
✅ Exported 'nse_gainers' to SQL successfully.
✅ Exported 'nse_loosers' to SQL successfully.
✅ Exported 'bse_gainers' to SQL successfully.
✅ Exported 'bse_loosers' to SQL successfully.
